In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json

from config import api_key
import pprint

import datetime as dt
from datetime import date, timedelta

In [2]:
url="http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31"
url

'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31'

In [ ]:
response=requests.get(url)
response

In [ ]:
weather_json=response.json()
weather_json


In [ ]:
weather_json['data']['weather'][0]['date']

In [ ]:
weather_json['data']['weather'][0]['maxtempC']

In [ ]:
weather_json['data']['weather'][0]['hourly'][0]['humidity']

In [ ]:
weather_json['data']['weather'][0]['hourly'][7]['windspeedKmph']

In [ ]:
# day 1
# daily avg windspeed in Kmph for hour=0 to 7
# daily total precipitation in MM for hour=0 to 7
# daily average cloud cover
windspeed = 0
total_precip = 0
cloud = 0
x = len(weather_json['data']['weather'][0]['hourly'])
for i in range(x):
    windspeed += int(weather_json['data']['weather'][0]['hourly'][i]['windspeedKmph'])
    total_precip += float(weather_json['data']['weather'][0]['hourly'][i]['precipMM'])
    cloud += int(weather_json['data']['weather'][0]['hourly'][i]['cloudcover'])
    humidity = int(weather_json['data']['weather'][0]['hourly'][i]['humidity'])

avg_windspeed = windspeed/x
avg_humidity = humidity/x
avg_cloud = cloud/x
print(avg_windspeed)
print(total_precip)
print(avg_cloud)
print(avg_humidity)

In [ ]:
url="http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31"
response=requests.get(url)

weather_json=response.json()
weather_json


In [3]:
# request one year of data
start_date = '2019-01-01'
end_date = '2019-12-31'
# Use the start and end date to create a range of dates
start_dates = pd.date_range(start_date, end_date, freq='MS')
start_dates = list(np.ravel(pd.to_datetime(start_dates).strftime("%Y-%m-%d")))

end_dates = pd.date_range(end = end_date, periods = 12, freq='M')
end_dates = list(np.ravel(pd.to_datetime(end_dates).strftime("%Y-%m-%d")))


In [ ]:
start_dates

In [ ]:
end_dates

In [ ]:
#url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date={start_dates[0]}&enddate={end_dates[0]}"
url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key="
dates = f"&q=New+York&format=json&date={start_dates[0]}&enddate={end_dates[0]}"
query_url = url + {api_key} + dates

response=requests.get(url)

weather_json=response.json()
# weather_json

In [4]:
# loop thru all months in 2019
nyc = 'New+York'
url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key="
city = f"&q={nyc}&format=json&date="

forecast_date = []
maxTempC = []
humidity = []
total_precip = []
avg_cloudcover = []
avg_windspeed = []


for dt in range(len(start_dates)):
    query_url = f"{url}{api_key}{city}{start_dates[dt]}&enddate={end_dates[dt]}"
    print(query_url)
    
# request for data
    response=requests.get(query_url)
    weather_json=response.json()

# loop through one month data
    days = len(weather_json['data']['weather'])
    for day in range(days):
        dates = weather_json['data']['weather'][day]['date'] # date
        maxtempc = weather_json['data']['weather'][day]['maxtempC'] # maxTempC
        
#       append to lists
        forecast_date.append(dates)
        maxTempC.append(maxtempc)

        windspeed = 0
        totalprecip = 0
        cloud = 0
        humid = 0
        x = len(weather_json['data']['weather'][day]['hourly'])
        for i in range(x):
            windspeed += int(weather_json['data']['weather'][day]['hourly'][i]['windspeedKmph'])
            totalprecip += float(weather_json['data']['weather'][day]['hourly'][i]['precipMM'])
            cloud += int(weather_json['data']['weather'][day]['hourly'][i]['cloudcover'])
            humid += int(weather_json['data']['weather'][day]['hourly'][i]['humidity'])

        avg_humidity = humid/x
        avg_cloud = cloud/x
        avg_wind_speed = windspeed/x
        
# append weather info to lists
        humidity.append(round(avg_humidity,3))
        total_precip.append(round(totalprecip,3))
        avg_cloudcover.append(round(avg_cloud,3))
        avg_windspeed.append(round(avg_wind_speed,3))

        print(f"{dates} {maxtempc} {windspeed} {cloud} {totalprecip} {humid}")

http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31
2019-01-01 12 133 374 26.9 609
2019-01-02 4 80 85 0.0 442
2019-01-03 5 78 359 1.0 517
2019-01-04 7 105 260 0.0 566
2019-01-05 6 98 697 27.800000000000004 569
2019-01-06 7 115 217 0.0 498
2019-01-07 3 62 327 0.0 423
2019-01-08 7 80 772 4.3999999999999995 662
2019-01-09 6 165 419 0.30000000000000004 544
2019-01-10 2 149 465 0.2 462
2019-01-11 -2 126 63 0.0 361
2019-01-12 0 55 288 0.0 417
2019-01-13 -1 83 628 0.4 428
2019-01-14 -1 94 255 0.0 399
2019-01-15 3 64 200 0.0 425
2019-01-16 4 82 254 0.0 509
2019-01-17 -1 77 438 0.0 341
2019-01-18 4 50 648 0.6 571
2019-01-19 2 72 449 1.8 560
2019-01-20 8 171 713 56.0 653
2019-01-21 -12 214 447 0.2 429
2019-01-22 -3 78 87 0.0 401
2019-01-23 5 110 588 0.2 575
2019-01-24 10 143 736 45.300000000000004 691
2019-01-25 1 132 103 0.0 489
2019-01-26 -1 82 248 0.0 392
2019-01-27 6 86 316 0.0 517
2019-01

2019-09-01 25 122 237 0.0 407
2019-09-02 28 134 557 15.4 523
2019-09-03 30 86 174 2.8 446
2019-09-04 31 141 142 0.1 487
2019-09-05 25 107 522 0.1 447
2019-09-06 23 129 671 0.0 444
2019-09-07 26 138 441 0.1 448
2019-09-08 26 94 337 0.0 398
2019-09-09 21 68 576 0.2 395
2019-09-10 24 94 315 0.0 558
2019-09-11 31 113 109 0.0 503
2019-09-12 28 100 496 7.8 491
2019-09-13 22 156 228 0.2 452
2019-09-14 23 139 689 0.2 524
2019-09-15 31 76 379 10.8 478
2019-09-16 28 93 313 0.6000000000000001 419
2019-09-17 25 115 80 0.0 401
2019-09-18 21 104 403 0.1 419
2019-09-19 26 92 0 0.0 302
2019-09-20 28 82 28 0.0 302
2019-09-21 30 67 50 0.0 376
2019-09-22 29 94 84 0.0 465
2019-09-23 30 147 276 3.8 502
2019-09-24 24 154 212 6.3 431
2019-09-25 27 109 8 0.0 385
2019-09-26 27 110 186 0.0 420
2019-09-27 26 89 58 0.0 352
2019-09-28 28 99 220 1.3 478
2019-09-29 26 110 194 0.8999999999999999 470
2019-09-30 21 93 325 0.0 428
http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a3194

In [5]:
#forecast_date
#maxTempC
#humidity
#total_precip
#avg_cloudcover
avg_windspeed

[16.625,
 10.0,
 9.75,
 13.125,
 12.25,
 14.375,
 7.75,
 10.0,
 20.625,
 18.625,
 15.75,
 6.875,
 10.375,
 11.75,
 8.0,
 10.25,
 9.625,
 6.25,
 9.0,
 21.375,
 26.75,
 9.75,
 13.75,
 17.875,
 16.5,
 10.25,
 10.75,
 9.375,
 11.875,
 23.625,
 19.5,
 10.125,
 8.75,
 6.125,
 8.625,
 12.625,
 7.625,
 8.0,
 18.75,
 19.75,
 8.125,
 7.75,
 17.375,
 17.0,
 12.875,
 11.375,
 13.625,
 10.75,
 11.125,
 10.125,
 9.125,
 12.625,
 9.25,
 7.75,
 11.875,
 24.625,
 15.75,
 12.375,
 7.875,
 9.25,
 10.25,
 7.625,
 15.875,
 12.75,
 17.375,
 13.625,
 8.75,
 9.125,
 15.875,
 15.75,
 13.125,
 9.75,
 10.625,
 16.875,
 23.5,
 16.5,
 9.375,
 9.125,
 11.5,
 19.625,
 24.5,
 29.375,
 16.5,
 10.625,
 17.625,
 12.75,
 13.75,
 13.5,
 17.125,
 20.875,
 21.125,
 13.0,
 22.0,
 14.75,
 15.125,
 8.375,
 10.875,
 14.625,
 10.625,
 18.25,
 13.125,
 16.75,
 13.25,
 11.75,
 28.875,
 23.375,
 13.125,
 19.5,
 22.125,
 26.25,
 11.75,
 12.375,
 9.875,
 17.625,
 9.375,
 15.875,
 25.75,
 14.125,
 12.5,
 9.0,
 14.75,
 7.75,
 10.75,
 5

In [8]:
weather_df = pd.DataFrame({"location_id": 4, "forecast_date": forecast_date,
                           "maxTempC": maxTempC, "humidity": humidity,
                           "total_precip": total_precip, "avg_cloudcover": avg_cloudcover,
                           "avg_windspeed": avg_windspeed
                          })


In [9]:
weather_df

,location_id,forecast_date,maxTempC,humidity,total_precip,avg_cloudcover,avg_windspeed
0,4,2019-01-01,12,76.125,26.9,46.750,16.625
1,4,2019-01-02,4,55.250,0.0,10.625,10.000
2,4,2019-01-03,5,64.625,1.0,44.875,9.750
3,4,2019-01-04,7,70.750,0.0,32.500,13.125
4,4,2019-01-05,6,71.125,27.8,87.125,12.250
...,...,...,...,...,...,...,...
360,4,2019-12-27,9,88.750,0.5,78.750,9.750
361,4,2019-12-28,10,72.000,0.0,20.000,10.375
362,4,2019-12-29,7,64.000,1.3,46.875,8.000
363,4,2019-12-30,8,93.125,50.8,99.875,21.000


In [ ]:
# loop through one month data
days = len(weather_json['data']['weather'])
for day in range(days):
    dates = weather_json['data']['weather'][day]['date'] # date
    maxtempC = weather_json['data']['weather'][day]['maxtempC'] # maxTempC
    
    windspeed = 0
    total_precip = 0
    cloud = 0
    x = len(weather_json['data']['weather'][day]['hourly'])
    for i in range(x):
        windspeed += int(weather_json['data']['weather'][day]['hourly'][i]['windspeedKmph'])
        total_precip += float(weather_json['data']['weather'][day]['hourly'][i]['precipMM'])
        cloud += int(weather_json['data']['weather'][day]['hourly'][i]['cloudcover'])
        humidity += int(weather_json['data']['weather'][day]['hourly'][i]['humidity'])
    
    avg_windspeed = round(windspeed/x,3)
    avg_humidity = round(humidity/x,3)
    avg_cloud = round(cloud/x,3)
    
    print(f"{dates} {maxtempC} {avg_windspeed} {avg_cloud} {total_precip} {avg_humidity}")

In [ ]:

df = pd.DataFrame()

In [ ]:
#loop through dates and perform a request for data on each
#max temp, percipitation, wind speed, cloudiness, lat, long
dates=[]
max_temp=[]
percip=[]
wind_speed=[]
cloudiness=[]
lat=[]
long=[]

for date in dates:
    response=requests.get(url).json()
    date.append(response[''])